In [1]:
!pip install -q transformers==4.57.3 datasets accelerate evaluate trl==0.26.2 protobuf sentencepiece
!pip install -q huggingface_hub tensorboard

print("\nDependencies installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00

Dependencies installed!


In [3]:
from huggingface_hub import login

login()

### Preparando tools e dataset

In [2]:
import json
from datasets import Dataset
from transformers.utils import get_json_schema


def pagamento(valor: float, metodo_pagamento: str):
    """
    Tool de pagamento

    Args:
        valor: valor numérico da transação em reais
        metodo_pagamento: método utilizado para pagamento (ex: "pix", "debito", "credito")

    Returns:
        dict: Dicionário contendo mensagem de confirmação e dados de pagamento
    """
    return {
        "mensagem": "Pagamento realizado com sucesso",
        "dados": {
            "valor": valor,
            "metodo_pagamento": metodo_pagamento
        }
    }


TOOLS = [
    get_json_schema(pagamento)
]

print("Tools defined:")
for tool in TOOLS:
    print(f"   - {tool['function']['name']}: {tool['function']['description'][:50]}...")

Tools defined:
   - pagamento: Tool de pagamento...


In [8]:
# STEP: Convert dataset to Google FunctionGemma format

import json
from datasets import Dataset

# FunctionGemma special tokens
START_TURN = ""
END_TURN = ""
START_DECL = ""
END_DECL = ""
START_CALL = ""
END_CALL = ""
ESCAPE = ""

# Function declaration in Google format (PIX payment)
FUNCTION_DECLARATIONS = f"""{START_DECL}declaration:pagamento{{description:{ESCAPE}Realiza um pagamento via Pix{ESCAPE},parameters:{{properties:{{valor:{{description:{ESCAPE}Valor do pagamento em reais{ESCAPE},type:{ESCAPE}NUMBER{ESCAPE}}},metodo_pagamento:{{description:{ESCAPE}Metodo de pagamento{ESCAPE},type:{ESCAPE}STRING{ESCAPE}}}}},required:[{ESCAPE}valor{ESCAPE},{ESCAPE}metodo_pagamento{ESCAPE}],type:{ESCAPE}OBJECT{ESCAPE}}}}}{END_DECL}"""

SYSTEM_PROMPT = f"""{START_TURN}developer
You are a model that can do function calling with the following functions
{FUNCTION_DECLARATIONS}
{END_TURN}
"""

def create_training_example(sample):
    """
    Creates training example in exact Google FunctionGemma format.
    """

    user_content = sample["user_content"]
    tool_name = sample["tool_name"]

    tool_args_raw = sample["tool_arguments"]

    if isinstance(tool_args_raw, str):
        tool_args = json.loads(tool_args_raw)
    elif isinstance(tool_args_raw, dict):
        tool_args = tool_args_raw
    else:
        raise ValueError(
            f"Formato inesperado para tool_arguments: {type(tool_args_raw)}"
        )

    # Build prompt (input)
    prompt = f"""{SYSTEM_PROMPT}{START_TURN}user
{user_content}
{END_TURN}
{START_TURN}model
"""

    # IMPORTANT:
    # Google FunctionGemma format does NOT quote strings inside call
    params_str = ",".join(
        f"{k}:{v}" for k, v in tool_args.items()
    )

    completion = f"{START_CALL}call:{tool_name}{{{params_str}}}{END_CALL}"

    return {"text": prompt + completion}


# Load and convert dataset
raw_data = []
with open(
    "/content/drive/MyDrive/function-gemma-tuned/machine_actions_extenso.jsonl",
    "r",
    encoding="utf-8"
) as f:
    for line in f:
        raw_data.append(json.loads(line.strip()))

print(f"Loaded {len(raw_data)} raw examples")

dataset = Dataset.from_list(raw_data)
dataset = dataset.map(
    create_training_example,
    remove_columns=dataset.features
)

# Split into train/test (80% / 20%)
dataset = dataset.train_test_split(
    test_size=0.2,
    shuffle=True,
    seed=42
)

print("\nDataset prepared:")
print(f"   Train: {len(dataset['train'])} examples")
print(f"   Test:  {len(dataset['test'])} examples")

# Show sample
print(f"\n{'='*60}")
print("Sample training example:")
print("="*60)
print(dataset['train'][0]['text'][:800])
print("...")


Loaded 255 raw examples


Map:   0%|          | 0/255 [00:00<?, ? examples/s]


Dataset prepared:
   Train: 204 examples
   Test:  51 examples

Sample training example:
developer
You are a model that can do function calling with the following functions
declaration:pagamento{description:Realiza um pagamento via Pix,parameters:{properties:{valor:{description:Valor do pagamento em reais,type:NUMBER},metodo_pagamento:{description:Metodo de pagamento,type:STRING}},required:[valor,metodo_pagamento],type:OBJECT}}

user
fazer pix de vinte e sete reais

model
call:pagamento{metodo_pagamento:pix,valor:27.0}
...


In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load FunctionGemma base model
BASE_MODEL = "google/functiongemma-270m-it"

print(f"Loading {BASE_MODEL}...")

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,      # 16-bit to save VRAM
    device_map="auto",                # Automatically load to GPU
    attn_implementation="eager"       # Without FlashAttention for compatibility
)

# Tokenizer converts text to tokens and back
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print(f"\nModel loaded!")
print(f"   Parameters: {model.num_parameters():,}")
print(f"   Memory: ~{model.num_parameters() * 2 / 1e9:.1f} GB (bfloat16)")
print(f"   Device: {model.device}")

Loading google/functiongemma-270m-it...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/13.8k [00:00<?, ?B/s]


Model loaded!
   Parameters: 268,098,176
   Memory: ~0.5 GB (bfloat16)
   Device: cuda:0


In [17]:

from trl import SFTConfig, SFTTrainer

# Output directory
OUTPUT_DIR = "/content/drive/MyDrive/function-gemma-tuned"
# =============================================================================
# Training configuration (based on official Google FunctionGemma cookbook)
# https://github.com/google-gemini/gemma-cookbook/blob/main/FunctionGemma/
# =============================================================================
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,

    # Dataset field with pre-formatted Google FunctionGemma format
    dataset_text_field="text",          # Use our pre-formatted text, NOT apply_chat_template

    # Training params (Google official uses 2 epochs, we use 5 for enum support)
    max_length=512,                    # Max sequence length in tokens
    packing=False,                      # Don't pack multiple examples into one sequence
    num_train_epochs=3,                 # Extended training for enum support (320 examples)
    per_device_train_batch_size=4,      # Batch size per GPU
    per_device_eval_batch_size=4,       # Eval batch size
    gradient_accumulation_steps=8,      # Effective batch size: 4 * 8 = 32

    learning_rate=5e-5,                 # Google official: 1e-5 (more conservative than 5e-5)
    lr_scheduler_type="cosine",         # Google official: cosine decay
    optim="adamw_torch_fused",          # Fused AdamW for faster training
    warmup_ratio=0.1,                   # 10% warmup steps

    # Logging and checkpoints
    logging_steps=10,                   # Log every 10 steps
    eval_strategy="epoch",              # Evaluate after each epoch
    save_strategy="epoch",              # Save checkpoint after each epoch

    # Memory optimization
    gradient_checkpointing=False,       # Trade compute for memory (enable if OOM)
    bf16=True,                          # Use bfloat16 for training

    # Output
    report_to="tensorboard",            # Log to TensorBoard
    push_to_hub=False,                  # Set to True to upload to HuggingFace
)

print("Training configuration (Google official params):")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   LR scheduler: {training_args.lr_scheduler_type}")
print(f"   Max length: {training_args.max_length}")
print(f"   Dataset field: {training_args.dataset_text_field}")


Training configuration (Google official params):
   Epochs: 3
   Batch size: 4
   Gradient accumulation: 8
   Effective batch size: 32
   Learning rate: 5e-05
   LR scheduler: SchedulerType.COSINE
   Max length: 512
   Dataset field: text


In [18]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
)

print("Starting training...")
print(f"   Train examples: {len(dataset['train'])}")
print(f"   Eval examples: {len(dataset['test'])}")
print(f"   Format: Google FunctionGemma (manual)")
print(f"   Estimated time: ~5 minutes on A100")
print("-" * 50)

# Train!
train_result = trainer.train()

print("\n" + "=" * 50)
print("Training complete!")
print(f"   Final loss: {train_result.training_loss:.4f}")

Adding EOS to train dataset:   0%|          | 0/204 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/204 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/204 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/51 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/51 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/51 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


Starting training...
   Train examples: 204
   Eval examples: 51
   Format: Google FunctionGemma (manual)
   Estimated time: ~5 minutes on A100
--------------------------------------------------


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,No log,0.318945,0.345729,24332.000000,0.945721
2,2.501100,0.165832,0.245112,48664.000000,0.961008
3,0.170400,0.151050,0.229305,72996.000000,0.959851



Training complete!
   Final loss: 1.2790


In [19]:
trainer.model.save_pretrained("/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned")
tokenizer.save_pretrained("/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned")

('/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned/tokenizer_config.json',
 '/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned/special_tokens_map.json',
 '/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned/chat_template.jinja',
 '/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned/tokenizer.model',
 '/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned/added_tokens.json',
 '/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned/tokenizer.json')

### Convert pytorch to gguf

1) Clone o llama.cpp:

`git clone https://github.com/ggerganov/llama.cpp`

`cd llama.cpp`

2) Instale as dependências:

`pip install -r requirements.txt`


3) Converter para GGUF

`python convert_hf_to_gguf.py \
  path/do/modelo_hf \
  --outfile model.gguf`



### FunctionGemma - Conversão para Q8

In [20]:
!/content/llama.cpp/convert_hf_to_gguf.py '/content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos-tuned' \
    --outfile /content/drive/MyDrive/function-gemma-tuned/functiongemma-pagamentos.Q8_0.gguf \
    --outtype q8_0

INFO:hf-to-gguf:Loading model: functiongemma-pagamentos-tuned
INFO:hf-to-gguf:Model architecture: Gemma3ForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> Q8_0, shape = {640, 262144}
INFO:hf-to-gguf:blk.0.attn_norm.weight,            torch.bfloat16 --> F32, shape = {640}
INFO:hf-to-gguf:blk.0.ffn_down.weight,             torch.bfloat16 --> Q8_0, shape = {2048, 640}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,             torch.bfloat16 --> Q8_0, shape = {640, 2048}
INFO:hf-to-gguf:blk.0.ffn_up.weight,               torch.bfloat16 --> Q8_0, shape = {640, 2048}
INFO:hf-to-gguf:blk.0.post_attention_norm.weight,  torch.bfloat16 --> F32, shape = {640}
INFO:hf-to-gguf:blk.0.post_ffw_norm.weight,        torch.bfloat16 --> F32, shape = {640}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,             torch.bfloat16